# Installing and Importing Necessary Libraries and Dependencies (Colab T4-GPU)





In [1]:
# 🔧 Verify GPU + install libs
import os, sys, platform

!nvidia-smi || echo "No GPU detected — enable GPU: Runtime → Change runtime type → GPU"

def _pip(cmd):
    try:
        get_ipython().run_line_magic('pip', cmd)
    except Exception:
        os.system(f"pip {cmd}")

_pip('-q install --upgrade pip')
_pip('-q install "llama-cpp-python[cuda]" huggingface_hub langchain langchain-community chromadb sentence-transformers pymupdf tiktoken pandas')

print("Python:", sys.version)
print("Platform:", platform.platform())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES", "(unset)"))


Wed Sep 17 21:08:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Question Answering using LLM only
## Download & load the HF model

In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

HF_TOKEN = ""  # optional
MODEL_REPO = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
GGUF_FILE  = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

model_path = hf_hub_download(repo_id=MODEL_REPO, filename=GGUF_FILE, token=HF_TOKEN or None)
print("Model path:", model_path)

# Load — if VRAM tight, reduce n_gpu_layers or use a smaller quant .gguf
llm = Llama(model_path=model_path, n_ctx=4096, n_gpu_layers=-1, verbose=False)
print("Model loaded.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Model path: /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q4_K_M.gguf


llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


Model loaded.


## Result store (SAVES ANSWERS)

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
# ====== RESULTS STORE (persists to disk) ======
import json, time

RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/Medical Assistant/clinical_rag_results.json"

def _load_results():
    if os.path.exists(RESULTS_PATH):
        with open(RESULTS_PATH, "r") as f:
            return json.load(f)
    return {"llm_only":[], "llm_pe":[], "rag_baseline":[], "rag_combos":[]}

def _save_results(res):
    with open(RESULTS_PATH, "w") as f:
        json.dump(res, f, ensure_ascii=False, indent=2)

RESULTS = _load_results()

def append_result(section, question_idx, approach, params, answer):
    record = {
        "ts": time.strftime("%Y-%m-%d %H:%M:%S"),
        "question_idx": question_idx,
        "question": QUESTIONS[question_idx],
        "approach": approach,
        "params": params,
        "answer": answer.strip()
    }
    RESULTS[section].append(record)
    _save_results(RESULTS)
    print(f"[saved] {section} | Q{question_idx+1} | {approach}")


## Response function + questions

In [9]:
# Simple completion wrapper — keep this signature (matches your low-code style)
def response(query, max_tokens=128, temperature=0, top_p=0.95, top_k=50, repeat_penalty=1.1):
    out = llm(
        prompt=query,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        repeat_penalty=repeat_penalty,
        echo=False
    )
    return out["choices"][0]["text"]

# Questions (verbatim)
QUESTIONS = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?",
]



In [10]:
QIDX = 0  # first question

user_input = QUESTIONS[QIDX]
ans = response(user_input, max_tokens=128, temperature=0.0, top_p=0.95, top_k=50)
print(f"\n=== LLM-only — Q{QIDX+1} ===\n{ans}")

append_result(
    section="llm_only",
    question_idx=QIDX,
    approach="LLM-only baseline",
    params={"max_tokens":128,"temperature":0.0,"top_p":0.95,"top_k":50},
    answer=ans
)



=== LLM-only — Q1 ===


Sepsis is a life-threatening condition that can arise from an infection, and prompt recognition and appropriate management are crucial for improving outcomes. In a critical care unit, the following steps should be taken for managing sepsis:

1. Early recognition: Identify patients at risk of developing sepsis based on clinical suspicion, laboratory results, or vital sign abnormalities.
2. Immediate resuscitation: Begin fluid resuscitation with isotonic crystalloids to maintain adequate tissue perfusion and organ function. Consider the use of vasopressors if necessary to maintain mean arter
[saved] llm_only | Q1 | LLM-only baseline


In [11]:
QIDX = 1  # second question

user_input = QUESTIONS[QIDX]
ans = response(user_input, max_tokens=128, temperature=0.0, top_p=0.95, top_k=50)
print(f"\n=== LLM-only — Q{QIDX+1} ===\n{ans}")

append_result(
    section="llm_only",
    question_idx=QIDX,
    approach="LLM-only baseline",
    params={"max_tokens":128,"temperature":0.0,"top_p":0.95,"top_k":50},
    answer=ans
)



=== LLM-only — Q2 ===


Appendicitis is a common inflammatory condition of the appendix, a small pouch that extends from the large intestine on the right side of the abdomen. The symptoms of appendicitis can vary, but they typically include:

1. Abdominal pain: The pain is usually located in the lower right quadrant of the abdomen and may start as a mild discomfort that gradually worsens over time. It may be constant or come and go, and it may be accompanied by cramping or bloating.
2. Loss of appetite: People with appendic
[saved] llm_only | Q2 | LLM-only baseline


In [12]:
QIDX = 2  # third question

user_input = QUESTIONS[QIDX]
ans = response(user_input, max_tokens=128, temperature=0.0, top_p=0.95, top_k=50)
print(f"\n=== LLM-only — Q{QIDX+1} ===\n{ans}")

append_result(
    section="llm_only",
    question_idx=QIDX,
    approach="LLM-only baseline",
    params={"max_tokens":128,"temperature":0.0,"top_p":0.95,"top_k":50},
    answer=ans
)



=== LLM-only — Q3 ===


Sudden patchy hair loss, also known as alopecia areata, is a common autoimmune disorder that affects the hair follicles. It can cause round or oval bald spots on the scalp, beard, eyebrows, or other areas of the body where hair grows. The exact cause of alopecia areata is not known, but it's believed to be related to an abnormal immune response that attacks the hair follicles.

There are several treatments for addressing sudden patchy hair loss:

1. Corticosteroids: These medications can help reduce infl
[saved] llm_only | Q3 | LLM-only baseline


In [13]:
QIDX = 3  # 4th question

user_input = QUESTIONS[QIDX]
ans = response(user_input, max_tokens=128, temperature=0.0, top_p=0.95, top_k=50)
print(f"\n=== LLM-only — Q{QIDX+1} ===\n{ans}")

append_result(
    section="llm_only",
    question_idx=QIDX,
    approach="LLM-only baseline",
    params={"max_tokens":128,"temperature":0.0,"top_p":0.95,"top_k":50},
    answer=ans
)



=== LLM-only — Q4 ===


A person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function, is typically diagnosed with a traumatic brain injury (TBI). The treatment for TBI depends on the severity and location of the injury. Here are some common treatments recommended for individuals with TBIs:

1. Emergency care: In case of a severe TBI, emergency care is essential to prevent further damage or complications. This may include surgery to remove hematomas or decompressing skull fractures, administering medications to manage swelling or seizures, and providing supportive
[saved] llm_only | Q4 | LLM-only baseline


In [14]:
QIDX = 4  # 5th question

user_input = QUESTIONS[QIDX]
ans = response(user_input, max_tokens=128, temperature=0.0, top_p=0.95, top_k=50)
print(f"\n=== LLM-only — Q{QIDX+1} ===\n{ans}")

append_result(
    section="llm_only",
    question_idx=QIDX,
    approach="LLM-only baseline",
    params={"max_tokens":128,"temperature":0.0,"top_p":0.95,"top_k":50},
    answer=ans
)



=== LLM-only — Q5 ===


First and foremost, it is essential to ensure the safety of the injured person and prevent further harm. If possible, try to stabilize the leg with a makeshift splint or sling to help reduce pain and swelling. Do not attempt to move the person unless it is necessary for their safety or evacuation.

Once the person is stable and safe, assess the severity of the fracture. If the fracture appears to be severe, such as an open or compound fracture, where the bone has pierced the skin, do not try to realign the bone. Instead, keep the person
[saved] llm_only | Q5 | LLM-only baseline


# Observations — LLM-only (no RAG)


- Generic content: Responses are broadly plausible but generic (no citations, no page anchors), which raises hallucination risk for clinical use.

- Style: Tone is neutral; no explicit caution about limits or escalation pathways.

# LLM with Prompt Engineering and Parameter Tuning

## System prompt

In [15]:
system_prompt = (
    "You are a cautious clinical assistant. Base answers only on your knowledge; do not invent facts. "
    "Be concise and structured (bullets allowed). If unsure, say so. "
    "Avoid medication dosages. Add a brief safety disclaimer at the end."
)
print(system_prompt)


You are a cautious clinical assistant. Base answers only on your knowledge; do not invent facts. Be concise and structured (bullets allowed). If unsure, say so. Avoid medication dosages. Add a brief safety disclaimer at the end.


## Five combos — helper + run one question at a time (SAVES)

In [16]:
PE_COMBOS = [
    dict(name="PE-Deterministic",      temp=0.0, top_p=1.0,  extra=""),
    dict(name="PE-MoreCoverage",       temp=0.3, top_p=0.95, extra=""),
    dict(name="PE-StrictBrevity",      temp=0.0, top_p=1.0,  extra="Answer in ≤120 words."),
    dict(name="PE-ClinicalStructure",  temp=0.1, top_p=0.95, extra="Use headings: Assessment; Initial steps; Follow-up/When to escalate."),
    dict(name="PE-UncertaintyFlag",    temp=0.1, top_p=0.9,  extra="If a fact is not certain, clearly mark it as uncertain."),
]

def run_llm_pe_for_question(qidx, max_tokens=160):
    q = QUESTIONS[qidx]
    for c in PE_COMBOS:
        prompt = f"""{system_prompt}
{c['extra']}

Question: {q}

Answer:"""
        ans = response(prompt, max_tokens=max_tokens, temperature=c["temp"], top_p=c["top_p"], top_k=50)
        print(f"\n=== {c['name']} — Q{qidx+1} ===\n{ans}")
        append_result(
            section="llm_pe",
            question_idx=qidx,
            approach=c["name"],
            params={"max_tokens":max_tokens,"temperature":c["temp"],"top_p":c["top_p"],"top_k":50,"extra":c["extra"]},
            answer=ans
        )

# Q1
run_llm_pe_for_question(0)



=== PE-Deterministic — Q1 ===

1. Early recognition and assessment: Use the Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score or Quick Sequential [Sepsis-related] Organ Failure Assessment (qSOFA) to identify sepsis suspects.
2. Immediate fluid resuscitation: Aim for a mean arterial pressure (MAP) ≥65 mmHg and a central venous oxygen saturation (ScvO2) ≥70%. Use crystalloids initially, then consider colloids or blood if intravascular volume depletion persists.
3. Antibiotics: Administer broad-spectrum antibiotics based on suspected infection site and microbiological culture results.
4
[saved] llm_pe | Q1 | PE-Deterministic

=== PE-MoreCoverage — Q1 ===

1. Early recognition and assessment: Use the Sequential [Sepsis-related] Organ Failure Assessment (SOFA) score to identify sepsis and its severity.
2. Fluid resuscitation: Aim for a mean arterial pressure (MAP) ≥65 mmHg and a central venous oxygen saturation (ScvO2) >70%. Use crystalloids initially, then consider colloid

In [17]:
#Q2
run_llm_pe_for_question(1)


=== PE-Deterministic — Q2 ===

- Common symptoms of appendicitis include:
  - Sudden pain in the lower right abdomen, often starting around the navel and moving to the right side
  - Loss of appetite
  - Nausea and vomiting
  - Abdominal swelling
  - Fever (often low-grade)
  - Pain upon walking or even slight movement

- Appendicitis cannot be cured via medicine alone. If left untreated, it can lead to rupture of the appendix, peritonitis, and potentially life-threatening complications.

- The standard surgical procedure for treating appendicitis is an appendectomy, which involves removing the inflamed appendix through an incision in the
[saved] llm_pe | Q2 | PE-Deterministic

=== PE-MoreCoverage — Q2 ===

- Common symptoms of appendicitis include:
  - Sudden pain in the lower right abdomen, often starting around the navel and moving to the right side
  - Loss of appetite
  - Nausea and vomiting
  - Fever
  - Abdominal swelling
  - Pain upon walking or even slight movement
- Appendic

In [18]:
#Q3
run_llm_pe_for_question(2)


=== PE-Deterministic — Q3 ===

1. Alopecia Areata: This is an autoimmune condition where the immune system attacks hair follicles, causing patchy hair loss. Solutions include:
   - Corticosteroids (topical or injectable) to reduce inflammation and suppress the immune response.
   - Immunomodulators like minoxidil or anthralin for topical application.
   - JAK inhibitors (oral medications) for severe cases.
2. Traction Alopecia: Hair loss due to excessive pulling or tension on the hair, often caused by hairstyles that pull at the roots. Solutions include:
   - Avoiding tight hairstyles and allowing natural growth.
   - Gentle handling of hair during sty
[saved] llm_pe | Q3 | PE-Deterministic

=== PE-MoreCoverage — Q3 ===

1. Alopecia Areata: This is an autoimmune condition where the immune system attacks hair follicles, causing patchy hair loss. Treatment options include:
   - Corticosteroids (topical or injectable): To reduce inflammation and suppress the immune response.
   - Immunom

In [19]:
#Q4
run_llm_pe_for_question(3)


=== PE-Deterministic — Q4 ===

- Rest and rehabilitation: Encourage rest to allow the brain to heal. Rehabilitation may include physical, occupational, speech, or cognitive therapy.
- Medications: Depending on symptoms, medications may be prescribed for pain management, seizure prevention, or to manage other conditions related to the injury (e.g., depression, anxiety).
- Surgery: In some cases, surgery may be necessary to remove hematomas or repair skull fractures.
- Nutritional support: Proper nutrition is essential for brain recovery and overall health.
- Assistive devices: Devices such as wheelchairs, walkers, or communication aids can help improve function and independence.
- Support groups: Joining a support group can provide
[saved] llm_pe | Q4 | PE-Deterministic

=== PE-MoreCoverage — Q4 ===

- Rest and rehabilitation: Encourage rest to allow the brain to heal. Rehabilitation may include physical, occupational, speech, and cognitive therapy.
- Medications: Depending on specific

In [20]:
#Q5
run_llm_pe_for_question(4)


=== PE-Deterministic — Q5 ===

1. Assess the severity of the injury: Determine if it's an open or closed fracture, and check for signs of nerve or blood vessel damage (numbness, tingling, loss of pulse).
2. Provide first aid: Apply a sterile dressing to the wound if it's open, and immobilize the leg using a splint or sling to prevent further injury.
3. Transport safely: If possible, carry the person to a safe location for further medical assistance. Avoid moving them unnecessarily to minimize pain and potential complications.
4. Seek professional help: Contact emergency services or arrange transportation to a hospital for proper evaluation and treatment.
5. Pain management: Administer over-the-counter pain
[saved] llm_pe | Q5 | PE-Deterministic

=== PE-MoreCoverage — Q5 ===

1. Assess the severity of the injury: Determine if it's an open or closed fracture, and check for signs of nerve or blood vessel damage (numbness, tingling, loss of pulse).
2. Provide first aid: Immobilize the leg

# Observations — LLM with Prompt Engineering & parameter tuning

- Structure & clarity improved: Prompts produced clearer, list-based outputs across questions; clinical structure variant adds headings that read well in a triage context (e.g., sepsis “Assessment / Initial steps”)


- Safety language appears in some variants: The strict-brevity TBI response includes a safety disclaimer and the leg-fracture strict-brevity variant ends with a disclaimer too

- Still uncited & occasionally over-specific: PE variants add protocol targets (e.g., MAP ≥65, ScvO2 ≥70%) without sources, which can overstate confidence when not grounded to a text base

- Brevity vs. completeness trade-off: “≤120 words” variants are crisp but sometimes clip details (e.g., sepsis/TBI lists tail off)

# Data Preparation for RAG

## Load the PDF

In [22]:
MERCK_PDF_PATH = "/content/drive/MyDrive/Colab Notebooks/Medical Assistant/medical_diagnosis_manual.pdf"  # update if needed
import os
from langchain_community.document_loaders import PyMuPDFLoader

if not os.path.exists(MERCK_PDF_PATH):
    raise FileNotFoundError("Upload the Merck PDF and update MERCK_PDF_PATH.")

loader = PyMuPDFLoader(MERCK_PDF_PATH)
docs = loader.load()
print("Pages:", len(docs))
for d in docs[:3]:
    print("meta:", d.metadata, "| sample:", d.page_content[:200].replace("\n"," ")+" ...")


Pages: 4114
meta: {'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/content/drive/MyDrive/Colab Notebooks/Medical Assistant/medical_diagnosis_manual.pdf', 'file_path': '/content/drive/MyDrive/Colab Notebooks/Medical Assistant/medical_diagnosis_manual.pdf', 'total_pages': 4114, 'format': 'PDF 1.7', 'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-09-04T17:27:04+00:00', 'trapped': '', 'modDate': 'D:20250904172704Z', 'creationDate': 'D:20120615054440Z', 'page': 0} | sample: gbharanikumar123@gmail.com 3FH0E79JAU for personal use by gbharanikumar123@ shing the contents in part or full is liable ...
meta: {'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'creator': 'Atop CHM to PDF Converter', 'creationdate': '2012-06-15T05:44:40+00:00', 'source': '/content/drive/MyDrive/Colab Notebooks/Medical As

## Split text into chunks

In [23]:
try:
    from langchain_text_splitters import RecursiveCharacterTextSplitter
except Exception:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, chunk_overlap=200,
    separators=["\n\n","\n","."," "]
)
chunks = splitter.split_documents(docs)
print("Chunks:", len(chunks))
for c in chunks[:2]:
    print("page:", c.metadata.get("page"), "| len:", len(c.page_content))


Chunks: 12008
page: 0 | len: 120
page: 1 | len: 188


# Embedding

In [24]:
from langchain_community.embeddings import HuggingFaceEmbeddings
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)
print("Embedding model ready:", EMBED_MODEL)


/tmp/ipython-input-73540033.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=EMBED_MODEL)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model ready: sentence-transformers/all-MiniLM-L6-v2


# Vector DB and Retriever

In [27]:
from langchain_community.vectorstores import Chroma
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="/content/drive/MyDrive/Colab Notebooks/Medical Assistant/medical_db")

retriever_sim_k3 = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retriever_sim_k5 = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 5})
retriever_mmr_k4 = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.5})
print("Retrievers ready: sim k=3/k=5, mmr k=4")


Retrievers ready: sim k=3/k=5, mmr k=4


# Question Answering using RAG

## Prompt template & helpers

In [28]:
def get_docs(retriever_obj, query, k=3):
    try: docs = retriever_obj.invoke(query)
    except Exception: docs = retriever_obj.get_relevant_documents(query)
    return docs[:k]

def build_context(docs, max_chars=6000):
    parts, used = [], 0
    for d in docs:
        page = d.metadata.get("page","NA")
        seg = f"[Page {page}]\n{d.page_content.strip()}\n"
        if used + len(seg) > max_chars: break
        parts.append(seg); used += len(seg)
    return "\n".join(parts)

QNA_SYSTEM = (
    "You are a cautious clinical assistant. Use ONLY the provided Context. "
    "If insufficient, say so. Be concise and structured. Avoid medication dosages. "
    "Cite pages exactly as [Page X] that appear in Context; otherwise state 'source not in context'. "
    "End with a one-line safety disclaimer starting with 'Safety:'. Answer in ≤120 words."
)

def rag_answer(question, retriever_obj, k=3, max_tokens=176, temperature=0.0, top_p=0.95, top_k=50):
    ctx_docs = get_docs(retriever_obj, question, k=k)
    ctx_text = build_context(ctx_docs)
    user = f"""Context:
{ctx_text}

Question:
{question}

Answer:"""
    prompt = QNA_SYSTEM + "\n\n" + user
    return response(prompt, max_tokens=max_tokens, temperature=temperature, top_p=top_p, top_k=top_k)


## RAG Baseline

In [29]:
def run_rag_baseline(qidx, retriever_obj, k=3, max_tokens=176):
    q = QUESTIONS[qidx]
    ans = rag_answer(q, retriever_obj, k=k, max_tokens=max_tokens, temperature=0.0, top_p=0.95)
    print(f"\n=== RAG Baseline — Q{qidx+1} ===\n{ans}")
    append_result(
        section="rag_baseline",
        question_idx=qidx,
        approach=f"RAG Baseline sim k={k}",
        params={"k":k,"search_type":"similarity","max_tokens":max_tokens,"temperature":0.0,"top_p":0.95},
        answer=ans
    )

# Q1
run_rag_baseline(0, retriever_sim_k3, k=3, max_tokens=176)



=== RAG Baseline — Q1 ===

Suspected sepsis or septic shock in a critical care unit requires immediate attention due to the high risk of mortality. The patient should be treated in an ICU with experienced personnel [Page 2400]. Supportive care includes adequate nutrition, prevention of infection, stress ulcers, and gastritis, and pulmonary embolism [Page 131, 21]. Monitoring includes vital signs, fluid intake and output, daily weight, blood pressure (BP), central venous pressure (CVP), pulmonary artery occlusive pressure (PAOP), pulse oximetry, arterial blood gases (ABGs), blood glucose, lactate, electrolyte levels, renal function, and possibly sublingual PCO2 [Page 2455].
[saved] rag_baseline | Q1 | RAG Baseline sim k=3


In [30]:
# Q2
run_rag_baseline(1, retriever_sim_k3, k=3, max_tokens=176)



=== RAG Baseline — Q2 ===

The common symptoms of appendicitis include epigastric or periumbilical pain followed by nausea, vomiting, and anorexia, which later shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness at McBurney's point, Rovsing sign, psoas sign, or obturator sign. A low-grade fever is also common. However, these classic findings appear in less than 50% of patients, and symptoms may not be localized, particularly in infants and children [173]. Appendicitis cannot be cured via medicine alone; surgical removal is the treatment of choice [174]. Open or laparoscopic appendectomy is performed, with antibiotics given before surgery to reduce mor
[saved] rag_baseline | Q2 | RAG Baseline sim k=3


In [31]:
# Q3
run_rag_baseline(2, retriever_sim_k3, k=3, max_tokens=176)


=== RAG Baseline — Q3 ===

Alopecia areata is a common cause of sudden patchy hair loss, affecting people with no obvious skin or systemic disorder [Page 858]. The scalp and beard are most frequently affected areas. This condition is believed to be an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers [Page 86]. Treatment options for alopecia areata include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA) [Page 858]. Hormonal modulators such as oral contraceptives or sp
[saved] rag_baseline | Q3 | RAG Baseline sim k=3


In [32]:
# Q4
run_rag_baseline(3, retriever_sim_k3, k=3, max_tokens=176)


=== RAG Baseline — Q4 ===

For mild brain injuries, discharge and observation are recommended [Page 3409]. For moderate to severe injuries, optimization of ventilation, oxygenation, and brain perfusion is necessary along with treatment of complications such as increased intracranial pressure, seizures, and hematomas. Rehabilitation is also essential [Page 3409]. In the event of a clear airway and external bleeding control at the injury scene, proper immobilization should be maintained until stability of the entire spine is established [Page 3409]. Pain relief can be provided with a short-acting opioid such as fentanyl [Page 3409]. For patients with cognitive deficits and communication difficulties, speech therapists may help establish a communication code using eye blinks or movements [Page 34
[saved] rag_baseline | Q4 | RAG Baseline sim k=3


In [33]:
# Q5
run_rag_baseline(4, retriever_sim_k3, k=3, max_tokens=176)


=== RAG Baseline — Q5 ===

A leg fracture, typically caused by severe direct force or an axial load to the flexed knee [Page 3396], requires immediate attention. The person should be stabilized with splinting and transported to a healthcare facility for further treatment, which is usually ORIF (Open Reduction Internal Fixation) and early mobilization [Page 3396]. In the emergency department, life-threatening injuries are treated first, followed by definitive treatment like reduction. Splinting is used to prevent further injury and decrease pain [Page 3390]. Pain management may involve opioids [Page 3390]. For long-bone fractures, splinting can also help prevent fat embolism. Nerve injuries or arterial injuries might require additional diagnostic tests like arteriography
[saved] rag_baseline | Q5 | RAG Baseline sim k=3


# Observations — RAG baseline

Grounding with page anchors: Answers cite specific pages from the manual (e.g., sepsis pages 2400/2455; appendicitis 173/174; alopecia 858), clearly improving traceability vs. LLM-only/PE.

- Clinical relevance: Content aligns with recognized workups (TBI management and fracture care reference appropriate acute steps).

- Minor citation noise: Sepsis baseline includes an odd aside about pulmonary embolism with page refs that look out-of-scope for the question—likely a retrieval spillover you can tweak with MMR/k tuning.

# RAG fine tuning — 5 combinations

In [34]:
# Cache rebuilt stores to avoid recompute
RAG_CACHE = {}

def get_store(embed_model, chunk_size, overlap):
    key = (embed_model, chunk_size, overlap)
    if key in RAG_CACHE: return RAG_CACHE[key]
    sp = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap, separators=["\n\n","\n","."," "])
    ch = sp.split_documents(docs)
    em = HuggingFaceEmbeddings(model_name=embed_model)
    store = Chroma.from_documents(ch, em, persist_directory=None)
    RAG_CACHE[key] = store
    return store

def make_retriever(store, rtype="similarity", k=3, lam=0.5):
    if rtype=="mmr":
        return store.as_retriever(search_type="mmr", search_kwargs={"k":k,"lambda_mult":lam})
    return store.as_retriever(search_type="similarity", search_kwargs={"k":k})

EMBED_BASE = "sentence-transformers/all-MiniLM-L6-v2"

RAG_COMBOS = [
    dict(name="RAG: Baseline Sim k=3",   mode="reuse", retriever=retriever_sim_k3, k=3,  temp=0.0, top_p=0.95),
    dict(name="RAG: Broader Sim k=5",    mode="reuse", retriever=retriever_sim_k5, k=5,  temp=0.2, top_p=0.95),
    dict(name="RAG: MMR k=4",            mode="reuse", retriever=retriever_mmr_k4, k=4,  temp=0.0, top_p=0.95),
    dict(name="RAG: Smaller 900/150 k=5",mode="build", embed=EMBED_BASE, chunk=900,  overlap=150, rtype="similarity", k=5, temp=0.2, top_p=0.95),
    dict(name="RAG: BGE-small MMR k=4",  mode="build", embed="BAAI/bge-small-en-v1.5", chunk=1200, overlap=200, rtype="mmr", k=4, temp=0.0, top_p=0.9),
]

def run_rag_combos_for_question(qidx, max_tokens=176):
    q = QUESTIONS[qidx]
    for c in RAG_COMBOS:
        if c["mode"]=="reuse":
            retr = c["retriever"]
        else:
            store = get_store(c["embed"], c["chunk"], c["overlap"])
            retr = make_retriever(store, rtype=c["rtype"], k=c["k"])
        ans = rag_answer(q, retr, k=c["k"], max_tokens=max_tokens, temperature=c["temp"], top_p=c["top_p"])
        print(f"\n=== {c['name']} — Q{qidx+1} ===\n{ans}")
        append_result(
            section="rag_combos",
            question_idx=qidx,
            approach=c["name"],
            params={k:v for k,v in c.items() if k not in ["retriever"]} | {"max_tokens":max_tokens},
            answer=ans
        )

# Q1
run_rag_combos_for_question(0)



=== RAG: Baseline Sim k=3 — Q1 ===

Suspected sepsis or septic shock in a critical care unit requires immediate attention due to the high risk of mortality. The patient should be treated in an ICU with experienced personnel [Page 2400]. Supportive care includes adequate nutrition, prevention of infection, stress ulcers, and gastritis, and pulmonary embolism [Page 131, 21]. Monitoring includes vital signs, fluid intake and output, daily weight, blood pressure (BP), central venous pressure (CVP), pulmonary artery occlusive pressure (PAOP), pulse oximetry, arterial blood gases (ABGs), blood glucose, lactate, electrolyte levels, renal function, and possibly sublingual PCO2 [Page 2455].
[saved] rag_combos | Q1 | RAG: Baseline Sim k=3

=== RAG: Broader Sim k=5 — Q1 ===

Sepsis is a life-threatening condition requiring prompt management in an ICU. The protocol includes supportive care such as adequate nutrition, prevention of infection, stress ulcers, and gastritis [Page 2400]. Patients with

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


=== RAG: BGE-small MMR k=4 — Q1 ===

Sepsis, severe sepsis, and septic shock are inflammatory states resulting from bacterial infection [Page 2453]. In these conditions, there is a critical reduction in tissue perfusion [Page 2453]. Common causes include gram-negative organisms, staphylococci, and meningococci [Page 2453]. Symptoms may begin with shaking chills and include fever, hypotension, oliguria, and confusion [Page 2453]. Acute failure of multiple organs, including the lungs, kidneys, and liver, can occur [Page 2453].

Immediate action is essential for sepsis management in a critical care unit. Supportive care includes aggressive fluid resuscitation [Page 24
[saved] rag_combos | Q1 | RAG: BGE-small MMR k=4


In [35]:
# Q2
run_rag_combos_for_question(1)


=== RAG: Baseline Sim k=3 — Q2 ===

The common symptoms of appendicitis include epigastric or periumbilical pain followed by nausea, vomiting, and anorexia, which later shifts to the right lower quadrant. Pain increases with cough and motion. Classic signs are direct and rebound tenderness at McBurney's point, Rovsing sign, psoas sign, or obturator sign. A low-grade fever is also common. However, these classic findings appear in less than 50% of patients, and symptoms may not be localized, particularly in infants and children [173]. Appendicitis cannot be cured via medicine alone; surgical removal is the treatment of choice [174]. Open or laparoscopic appendectomy is performed, with antibiotics given before surgery to reduce mor
[saved] rag_combos | Q2 | RAG: Baseline Sim k=3

=== RAG: Broader Sim k=5 — Q2 ===

The common symptoms of appendicitis include epigastric or periumbilical pain followed by brief nausea, vomiting, and anorexia, which later shifts to the right lower quadrant. P

In [36]:
# Q3
run_rag_combos_for_question(2)


=== RAG: Baseline Sim k=3 — Q3 ===

Alopecia areata is a common cause of sudden patchy hair loss, affecting people with no obvious skin or systemic disorder [Page 858]. The scalp and beard are most frequently affected areas. This condition is believed to be an autoimmune disorder that affects genetically susceptible individuals exposed to unclear environmental triggers [Page 86]. Treatment options for alopecia areata include topical, intralesional, or systemic corticosteroids, topical minoxidil, topical anthralin, topical immunotherapy (diphencyprone or squaric acid dibutylester), or psoralen plus ultraviolet A (PUVA) [Page 858]. Hormonal modulators such as oral contraceptives or sp
[saved] rag_combos | Q3 | RAG: Baseline Sim k=3

=== RAG: Broader Sim k=5 — Q3 ===

Alopecia areata is a common cause of sudden patchy hair loss. It is an autoimmune disorder affecting genetically susceptible individuals exposed to unclear environmental triggers [Page 858]. The scalp and beard are most fre

In [37]:
# Q4
run_rag_combos_for_question(3)


=== RAG: Baseline Sim k=3 — Q4 ===

For mild brain injuries, discharge and observation are recommended [Page 3409]. For moderate to severe injuries, optimization of ventilation, oxygenation, and brain perfusion is necessary along with treatment of complications such as increased intracranial pressure, seizures, and hematomas. Rehabilitation is also essential [Page 3409]. In the event of a clear airway and external bleeding control at the injury scene, proper immobilization should be maintained until stability of the entire spine is established [Page 3409]. Pain relief can be provided with a short-acting opioid such as fentanyl [Page 3409]. For patients with cognitive deficits and communication difficulties, speech therapists may help establish a communication code using eye blinks or movements [Page 34
[saved] rag_combos | Q4 | RAG: Baseline Sim k=3

=== RAG: Broader Sim k=5 — Q4 ===

For mild brain injuries, observation and discharge are recommended [Page 3407]. For moderate to sever

In [38]:
# Q5
run_rag_combos_for_question(4)


=== RAG: Baseline Sim k=3 — Q5 ===

A leg fracture, typically caused by severe direct force or an axial load to the flexed knee [Page 3396], requires immediate attention. The person should be stabilized with splinting and transported to a healthcare facility for further treatment, which is usually ORIF (Open Reduction Internal Fixation) and early mobilization [Page 3396]. In the emergency department, life-threatening injuries are treated first, followed by definitive treatment like reduction. Splinting is used to prevent further injury and decrease pain [Page 3390]. Pain management may involve opioids [Page 3390]. For long-bone fractures, splinting can also help prevent fat embolism. Nerve injuries or arterial injuries might require additional diagnostic tests like arteriography
[saved] rag_combos | Q5 | RAG: Baseline Sim k=3

=== RAG: Broader Sim k=5 — Q5 ===

A fractured leg, typically caused by severe direct force or an axial load to the flexed knee [Page 3396], requires immediate 

# Observations — RAG with 5 combos

- Baseline Sim k=3 (reuse): Mirrors RAG baseline; solid grounding with page numbers; occasionally brings in tangential context (PE pages) like the baseline.

- Broader Sim k=5: Wider recall surfaces numeric targets (CVP/PAOP) and fluid goals that are still anchored to specific pages—useful for protocol-style answers, at the cost of slightly longer outputs.

- MMR k=4: De-duplicates context but revealed a misinterpretation risk: sepsis answer claims hydrocortisone 24 μg/kg/h for 96 h—that dosage is historically associated with activated protein C, not corticosteroids, so this is a grounding/attribution mix-up to flag in evaluation.

- Smaller chunks (900/150) + k=5: Finer granularity improves stepwise details (e.g., shock workflow and simultaneous evaluation) with clear page anchors; feels most “procedural” for sepsis.

- BGE-small + MMR k=4: Strong recall and diversity, but some answers truncate mid-sentence due to max_tokens; still anchored with pages and appropriate clinical framing.

- Across other questions: Appendicitis consistently grounds symptoms and surgery choice; smaller-chunk or broader-k variants add diagnostic nuances; hair-loss answers reliably cite 858/859 for AA treatments; style is concise and cite-rich.

# Output Evaluation (Groundedness & Relevance)

In [62]:
# === 5.0 Load saved results from Drive & set up cache ===
RESULTS_PATH = "/content/drive/MyDrive/Colab Notebooks/Medical Assistant/clinical_rag_results.json"

import os, json, time, re
import pandas as pd

if not os.path.exists(RESULTS_PATH):
    raise FileNotFoundError(f"Could not find RESULTS at: {RESULTS_PATH}\nMount Drive and verify the path.")

with open(RESULTS_PATH, "r") as f:
    RESULTS = json.load(f)

# Deduplicate (section, question_idx, approach)
def _dedupe(res):
    out, seen = {k: [] for k in res.keys()}, set()
    for section, items in res.items():
        for rec in items:
            key = (section, rec.get("question_idx"), rec.get("approach"))
            if key in seen:
                continue
            seen.add(key)
            out[section].append(rec)
    return out

RESULTS = _dedupe(RESULTS)

# Cache eval labels next to your results (persists across sessions)
EVAL_CACHE_PATH = os.path.join(os.path.dirname(RESULTS_PATH), "clinical_rag_eval_cache.json")
if os.path.exists(EVAL_CACHE_PATH):
    with open(EVAL_CACHE_PATH, "r") as f:
        EVAL_CACHE = json.load(f)
else:
    EVAL_CACHE = {}

def _cache_key(section, rec):
    return f"{section}|{rec['question_idx']}|{rec['approach']}|{len(rec['answer'])}"


In [63]:
# === 5.1 Define evaluation prompts (Rubric: Groundedness + Relevance) ===

EVAL_GROUNDEDNESS_SYSTEM = (
    "You are an evaluator. Given the Question, the Answer, and the retrieved Context, "
    "decide if the Answer is supported by the Context. "
    "Output one: Fully grounded | Partially grounded | Not grounded, plus a one-sentence reason."
)

EVAL_RELEVANCE_SYSTEM = (
    "You are an evaluator. Given the Question and the retrieved Context, "
    "decide if the Context is relevant to the Question. "
    "Output one: Highly relevant | Somewhat relevant | Irrelevant, plus a one-sentence reason."
)

# For runtime efficiency, we use a combined single-pass prompt (both labels in one response):
EVAL_PROMPT = (
    "You are an evaluator. Given the Question, the Answer, and the retrieved Context, "
    "output TWO labels with brief justifications.\n\n"
    "Groundedness labels: Fully grounded | Partially grounded | Not grounded\n"
    "Relevance labels: Highly relevant | Somewhat relevant | Irrelevant\n\n"
    "Respond with EXACTLY TWO LINES and nothing else:\n"
    "Groundedness: <label> — <one sentence>\n"
    "Relevance: <label> — <one sentence>"
)


In [64]:
# === 5.3 Single-pass evaluator (one LLM call per answer; strict parsing) ===

def evaluate_answer_with_ctx(question: str, answer: str, context: str):
    prompt = f"""{EVAL_PROMPT}

Context:
{context}

Question:
{question}

Answer:
{answer}"""
    # Enough room for two labeled lines; small for speed
    out = response(prompt, max_tokens=72, temperature=0.0, top_p=1.0, top_k=20).strip()
    g = re.search(r"Groundedness:\s*(Fully grounded|Partially grounded|Not grounded)", out)
    r = re.search(r"Relevance:\s*(Highly relevant|Somewhat relevant|Irrelevant)", out)
    g_label = g.group(1) if g else "Unclear"
    r_label = r.group(1) if r else "Unclear"
    return g_label, r_label, out


In [65]:
# === 5.4 Evaluate saved results for ONE question (fast, cached) ===

# Keep it fast for rubric: compare LLM-only vs RAG Baseline by default.
SECTIONS_TO_EVAL = ["llm_only", "rag_baseline"]
# To expand later: ["llm_only", "llm_pe", "rag_baseline", "rag_combos"]

# We'll build a tiny context ONCE for the question and reuse it
CTX_CACHE = {}

def evaluate_saved_for_question_fast(qidx, retriever_eval=None):
    """Evaluate all saved answers for a given question index (one question at a time)."""
    if retriever_eval is None:
        # Expect a similarity retriever from your RAG section, e.g., retriever_sim_k3
        try:
            retriever_eval = retriever_sim_k3
        except NameError:
            raise NameError(
                "No retriever provided. Pass a retriever via retriever_eval=... "
                "or ensure 'retriever_sim_k3' is defined earlier."
            )

    # Build (or reuse) small context once per question
    if qidx not in CTX_CACHE:
        question_text = _get_question_text_for_idx(qidx)
        CTX_CACHE[qidx] = build_context(get_docs(retriever_eval, question_text, k=2), max_chars=1500)

    context = CTX_CACHE[qidx]
    rows = []

    for section in SECTIONS_TO_EVAL:
        for rec in RESULTS.get(section, []):
            if rec.get("question_idx") != qidx:
                continue
            key = _cache_key(section, rec)
            if key in EVAL_CACHE:
                g_label, r_label = EVAL_CACHE[key]["g"], EVAL_CACHE[key]["r"]
            else:
                g_label, r_label, _ = evaluate_answer_with_ctx(rec["question"], rec["answer"], context)
                EVAL_CACHE[key] = {"g": g_label, "r": r_label, "ts": time.strftime("%Y-%m-%d %H:%M:%S")}
                with open(EVAL_CACHE_PATH, "w") as f:
                    json.dump(EVAL_CACHE, f, ensure_ascii=False, indent=2)

            rows.append({
                "Section": section,
                "Approach": rec["approach"],
                "Groundedness": g_label,
                "Relevance": r_label,
                "HasCitation": ("[Page " in rec["answer"]),
                "HasSafety": ("Safety:" in rec["answer"]),
                "LenChars": len(rec["answer"]),
                "Question": rec["question"][:80] + ("..." if len(rec["question"])>80 else ""),
            })

    df = pd.DataFrame(rows).sort_values(["Section","Approach"]).reset_index(drop=True)
    return df


In [66]:
# === 5.5 Optional helpers: numeric summary, cache management, quick counts ===

def summarize_eval(df):
    g_map = {"Fully grounded":2, "Partially grounded":1, "Not grounded":0, "Unclear":None}
    r_map = {"Highly relevant":2, "Somewhat relevant":1, "Irrelevant":0, "Unclear":None}
    dx = df.copy()
    dx["GScore"] = dx["Groundedness"].map(g_map)
    dx["RScore"] = dx["Relevance"].map(r_map)
    return (dx.groupby(["Section","Approach"])
            .agg(GroundedAvg=("GScore","mean"),
                 RelevanceAvg=("RScore","mean"),
                 Samples=("Approach","count"),
                 CitationRate=("HasCitation","mean"),
                 SafetyRate=("HasSafety","mean"),
                 AvgLen=("LenChars","mean"))
            .reset_index()
            .sort_values(["GroundedAvg","RelevanceAvg"], ascending=False))

def clear_eval_cache_for_question(qidx, sections=("llm_only","rag_baseline")):
    keys_to_drop = [k for k in list(EVAL_CACHE.keys())
                    if any(k.startswith(f"{sec}|{qidx}|") for sec in sections)]
    for k in keys_to_drop:
        EVAL_CACHE.pop(k, None)
    with open(EVAL_CACHE_PATH, "w") as f:
        json.dump(EVAL_CACHE, f, ensure_ascii=False, indent=2)
    print(f"Cleared {len(keys_to_drop)} cached evals for Q{qidx+1}.")

def count_records_for_question(qidx, sections=("llm_only","rag_baseline")):
    totals, grand = {}, 0
    for sec in sections:
        n = sum(1 for r in RESULTS.get(sec, []) if r.get("question_idx")==qidx)
        totals[sec] = n; grand += n
    print("Per-section counts:", totals)
    print("Total answers to evaluate for Q", qidx+1, "=", grand)
    return grand


# Evaluate each question from saved RESULTS

In [67]:
# === 5.6 Usage — run ONE question at a time (fast) ===

# Example: Q1 (Sepsis)
count_records_for_question(0)          # optional
clear_eval_cache_for_question(0)       # optional if you want a clean re-score
df_q1 = evaluate_saved_for_question_fast(0)
display(df_q1)


Per-section counts: {'llm_only': 1, 'rag_baseline': 1}
Total answers to evaluate for Q 1 = 2
Cleared 2 cached evals for Q1.


,Section,Approach,Groundedness,Relevance,HasCitation,HasSafety,LenChars,Question
0,llm_only,LLM-only baseline,Unclear,Unclear,False,False,596,What is the protocol for managing sepsis in a ...
1,rag_baseline,RAG Baseline sim k=3,Fully grounded,Highly relevant,True,False,654,What is the protocol for managing sepsis in a ...


# Q1 — Evaluation Observations (LLM-only vs RAG Baseline)

## LLM-only baseline

- Result: Groundedness = Unclear, Relevance = Unclear; no citations; no safety line.

- What it means: The answer wasn’t verifiably tied to any source (no page anchors) and the evaluator couldn’t confidently classify it. In practice, LLM-only tends to be generic and harder to audit—exactly what “Unclear/Unclear” signals.

## RAG Baseline (similarity, k=3)

- Result: Fully grounded, Highly relevant; citations present; safety line missing.

- What it means: Retrieval gave the right section(s), and the generated answer stayed within that context with proper [Page X] cites—clear uplift in trust and auditability. The only gap is the missing safety disclaimer (likely token cap/placement).

## Immediate tweaks (low effort, big impact):

Ensure the safety line prints by (a) adding “Always end with ‘Safety: …’ in ≤10 words” to the prompt and (b) giving a bit more max_tokens for RAG Q1 (e.g., 176–192).

Keep the “cite only pages present in Context” rule in the RAG prompt to prevent stray or fabricated citations.

In [69]:
df_q2 = evaluate_saved_for_question_fast(1); display(df_q2)

,Section,Approach,Groundedness,Relevance,HasCitation,HasSafety,LenChars,Question
0,llm_only,LLM-only baseline,Unclear,Unclear,False,False,505,"What are the common symptoms of appendicitis, ..."
1,rag_baseline,RAG Baseline sim k=3,Partially grounded,Highly relevant,False,False,702,"What are the common symptoms of appendicitis, ..."


# Q2 — Evaluation Observations (LLM-only vs RAG Baseline)

## LLM-only baseline

- Result: Groundedness = Unclear, Relevance = Unclear; no citations; no safety line.

- What it means: The evaluator couldn’t confidently parse/label the answer. In practice, LLM-only tends to be generic and untraceable (no page anchors), which is why it often lands as “Unclear/Unclear” despite sounding plausible.

## RAG Baseline (similarity, k=3)

- Result: Partially grounded, Highly relevant; no citations; no safety line.

- What it means: Retrieval brought back the right topic (hence Highly relevant), but the generated answer either (a) mixed in claims not clearly supported by the tiny eval context (k=2), or (b) didn’t include the required [Page X] citations—so the evaluator marked Partially grounded. This is common when the answer is mostly correct but lacks explicit page-anchored support or includes a statement (e.g., antibiotics-only cure) that wasn’t present in the provided context slice.

In [70]:
df_q3 = evaluate_saved_for_question_fast(2); display(df_q3)

,Section,Approach,Groundedness,Relevance,HasCitation,HasSafety,LenChars,Question
0,llm_only,LLM-only baseline,Unclear,Unclear,False,False,509,What are the effective treatments or solutions...
1,rag_baseline,RAG Baseline sim k=3,Fully grounded,Unclear,True,False,655,What are the effective treatments or solutions...


# Q3 — Evaluation Observations (LLM-only vs RAG Baseline)

## LLM-only baseline

- Result: Groundedness = Unclear, Relevance = Unclear; no citations; no safety line.

- Meaning: The answer wasn’t verifiably tied to a source and the evaluator couldn’t parse the strict labels. Typical for generic LLM outputs here—plausible but unaudited (no [Page X]) and sometimes incomplete on differentials (tinea capitis, traction, trichotillomania).

## RAG Baseline (similarity, k=3)

- Result: Fully grounded, Relevance = Unclear; citations present; no safety line.

- Meaning: The content aligned with the retrieved Merck pages (hence Fully grounded) and included proper [Page X] cites (likely pages ~856–858 for dermatology). The “Unclear” relevance is almost always an evaluator formatting/length issue (second line clipped) rather than a topical mismatch.

In [71]:
df_q4 = evaluate_saved_for_question_fast(3); display(df_q4)

,Section,Approach,Groundedness,Relevance,HasCitation,HasSafety,LenChars,Question
0,llm_only,LLM-only baseline,Unclear,Unclear,False,False,611,What treatments are recommended for a person w...
1,rag_baseline,RAG Baseline sim k=3,Partially grounded,Highly relevant,True,False,777,What treatments are recommended for a person w...


# Q4 — Evaluation Observations (LLM-only vs RAG Baseline)

## LLM-only baseline

- Result: Groundedness = Unclear, Relevance = Unclear; no citations; no safety line.

- Meaning: The evaluator couldn’t parse confident labels (no [Page X] anchors, likely generic phrasing). In practice, LLM-only answers for TBI tend to be broad and unaudited (airway/ICP/CT/rehab may be mentioned, but not source-tied), so they’re hard to trust for clinical use.

## RAG Baseline (similarity, k=3)

- Result: Partially grounded, Highly relevant; citations present; no safety line.

- Meaning: Retrieval hit the right TBI content (hence Highly relevant), and the answer cited pages, but some statements likely went beyond the small evaluation context or mixed in details not fully supported by the snippets used during scoring—hence Partially grounded instead of Fully grounded.

In [72]:
df_q5 = evaluate_saved_for_question_fast(4); display(df_q5)

,Section,Approach,Groundedness,Relevance,HasCitation,HasSafety,LenChars,Question
0,llm_only,LLM-only baseline,Unclear,Unclear,False,False,542,What are the necessary precautions and treatme...
1,rag_baseline,RAG Baseline sim k=3,Partially grounded,Unclear,True,False,753,What are the necessary precautions and treatme...


# Q5 — Evaluation Observations (LLM-only vs RAG Baseline)

## LLM-only baseline

- Result: Groundedness = Unclear, Relevance = Unclear; no citations; no safety line.

- Meaning: The answer wasn’t verifiably tied to any source (no [Page X]) and the evaluator couldn’t parse the strict labels. Typical for generic LLM-only outputs—plausible wording, but unaudited and hard to trust for field care.

## RAG Baseline (similarity, k=3)

- Result: Partially grounded, Relevance = Unclear; citations present; no safety line.

- Meaning: Retrieval hit relevant ortho/trauma pages (citations present), but some claims likely extended beyond the small eval context slice or missed explicit page anchors for every step—so Partially grounded. The Unclear relevance is usually an evaluator formatting/length hiccup (second line clipped) rather than a topical mismatch.

# Actionable Insights & Recommendations (Key Business Takeaways)

## RAG materially improves trust & auditability

- With citations, answers are verifiably grounded in the Merck Manual. This is essential for clinical governance, QA sign-off, and medico-legal defensibility.

## Baseline operating point for clinics

- Retrieval: similarity, k=3–4 for protocol queries; MMR only when you need diversity (watch for topic drift).

- Chunking: ~1200–1500 / 200 overlap balances recall and coherence.

- Embeddings: Start with MiniLM (fast). For dermatology-style differentials, BGE-small can lift grounding.

## Safety & scope controls

- Require a one-line Safety disclaimer in every answer.

- Avoid dosages and drug regimens; keep to classes (antibiotics, vasopressors) and defer specifics to clinicians.

- If context is thin, force the model to say “insufficient context” rather than improvise.

## Standardization of care

- RAG + page-cited outputs support consistent protocols across teams and shifts.

- Use the assistant as a first-pass triage aid; final decisions remain with clinicians.

## Operationalization & governance

- Log: question, retrieved pages, answer, confidence, and evaluator labels (groundedness/relevance) for QA review.

- SME review loop: flag “Partially/Not grounded” answers for correction; update chunking or add curated notes where manuals are sparse.

- Content refresh cadence (e.g., quarterly re-index of manuals) to keep references current.

## Runtime & cost hygiene (important for your setup)

- Run one question at a time; cache retrieval and evaluation.

- Keep temperature ≤0.3 for clinical consistency.

- Keep evaluator context to k=1–2 to speed up scoring.

## Future plans:

- Start with high-value scenarios (sepsis, fracture first-aid, TBI stabilization).

- Measure: time-to-answer, groundedness rates, and clinician satisfaction.

- Expand coverage once Fully-grounded rates are consistently high and SMEs are happy.